In [1]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import transforms

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [53]:
class TrainDataset(Dataset):
    def __init__(self, path, image_folder, label_file, transform):
        df = pd.read_csv(path + label_file)
        classes = df.labels.astype(str)
        
        self.image_paths = [path + image_folder + f for f in df.image]
        self.cls2idx = {c:i for i, c in enumerate(sorted(classes.unique()))}
        self.idx2cls = list(sorted(classes.unique()))
        self.y = classes.map(self.cls2idx).to_numpy()
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, i):
        img = Image.open(self.image_paths[i]).convert("RGB")
        img = self.transform(img)
        label = torch.tensor(self.y[i], dtype=torch.long)
        return img, label

class TestDataset(Dataset):
    def __init__(self, path, folder, transform):
        self.path = path + folder 
        self.image_names = [f for f in os.listdir(self.path)]
        self.transform = transform

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, i):
        img = Image.open(self.path + self.image_names[i]).convert("RGB")
        img = self.transform(img)

        return img, self.image_names[i]

In [54]:
from sklearn.model_selection import StratifiedShuffleSplit
from torch.utils.data import Subset

In [55]:

import kornia as K
from kornia.augmentation import AugmentationSequential, RandomRotation, RandomVerticalFlip

aug = AugmentationSequential(
    RandomRotation(degrees=90),
    RandomVerticalFlip(p=0.3),
    data_keys=["input"],         
    same_on_batch=False,
).to(device)

train_tfms = transforms.Compose([transforms.Resize((224, 224)),
                                 transforms.ToTensor(),
                                 transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
                                ])


test_tfms = transforms.Compose([transforms.Resize((224, 224)),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
                               ])

PATH = '/kaggle/input/plant-pathology-2021-fgvc8/'

train_dataset = TrainDataset(PATH, 'train_images/', 'train.csv', train_tfms)
val_dataset = TrainDataset(PATH, 'train_images/', 'train.csv', train_tfms)
test_dataset = TestDataset(PATH, 'test_images/', test_tfms)


y = train_dataset.y  
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, val_idx = next(sss.split(np.zeros(len(y)), y))

train_data = Subset(train_dataset, train_idx)  
val_data = Subset(val_dataset,   val_idx)   

batch_size = 256
num_workers = 4   # start low; increase only if GPU starves
prefetch_factor = 4

train_loader = DataLoader(
    train_data, batch_size=batch_size, shuffle=True,
    num_workers=num_workers, pin_memory=True, persistent_workers=True, 
    prefetch_factor=prefetch_factor
)
val_loader = DataLoader(
    val_data, batch_size=batch_size,
    num_workers=num_workers, pin_memory=True, persistent_workers=True, 
    prefetch_factor=prefetch_factor
)
test_loader = DataLoader(test_dataset, batch_size=batch_size,
    num_workers=num_workers, pin_memory=True, persistent_workers=True, 
                         prefetch_factor=prefetch_factor)



torch.backends.cudnn.benchmark = True  # once, after imports

In [56]:
from collections import OrderedDict  
from torch import nn, optim
from torchvision.models import resnet18

In [57]:
model = resnet18(pretrained=True)

model.fc = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(512, 128)),
    ('relu1', nn.ReLU()),
    ('droupout1', nn.Dropout(0.2)),
    ('fc2', nn.Linear(128, 12))
]))

model = model.to(device)
# model = nn.DataParallel(model, device_ids=[0,1])

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=1e-3) #model.module.fc.parameters() if use both gpus
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min')

scaler = torch.amp.GradScaler('cuda')

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Traceback (most recent call last):
  File "/usr/lib/python3.11/multiprocessing/queues.py", line 239, in _feed
    reader_close()
  File "/usr/lib/python3.11/multiprocessing/connection.py", line 178, in close
    self._close()
  File "/usr/lib/python3.11/multiprocessing/connection.py", line 377, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor


In [37]:
from tqdm import tqdm

In [44]:
for epoch in range(1):
    epoch_loss = 0
    correct_count_train = 0
    for ii, (images, labels) in tqdm(enumerate(train_loader), total=len(train_loader)):
        images_gpu = images.to(device, non_blocking=True)
        labels_gpu = labels.to(device, non_blocking=True)

        images_gpu = aug(images_gpu) #augmentations with kornia
        
        optimizer.zero_grad()
        with torch.amp.autocast('cuda'):  
            logits = model(images_gpu)
            loss = criterion(logits, labels_gpu)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        correct_count_train += (torch.argmax(logits, dim=-1) == labels_gpu).sum()
        epoch_loss += loss
    train_acc = correct_count_train/len(train_data)
    print(f'epoch {epoch}, train acc = {train_acc}')
        
    with torch.no_grad():
        model.eval()
        val_loss = 0
        correct_pred_count = 0
        for iii, (val_images, val_labels) in tqdm(enumerate(val_loader), total=len(val_loader)):
            val_images_gpu = val_images.to(device, non_blocking=True)
            val_labels_gpu = val_labels.to(device, non_blocking=True)
            with torch.amp.autocast('cuda'):
                val_logits = model(val_images_gpu)
                val_loss += criterion(val_logits, val_labels_gpu)
            correct_pred_count += (torch.argmax(val_logits, dim=-1) == val_labels_gpu).sum()
        val_acc = correct_pred_count / len(val_data) 
    model.train()
    scheduler.step(val_loss)
    print(f'epoch {epoch} train loss = {epoch_loss}, val loss = {val_loss}, val acc = {val_acc}')
        

  0%|          | 0/59 [00:02<?, ?it/s]


KeyboardInterrupt: 

In [59]:
import pandas

model.eval()
all_files, all_idx = [], []
for test_image, file_names in test_loader:
    test_image = test_image.to(device)
    preds = torch.argmax(model(test_image), dim=-1).cpu().tolist()
    all_idx.extend(preds)
    all_files.extend(file_names)

df = pd.DataFrame({'image': all_files, 'lables': [train_dataset.idx2cls[i] for i in all_idx]})
df.to_csv('submission.csv', index=False)
df.head(5)

,image,lables
0,ad8770db05586b59.jpg,scab
1,c7b03e718489f3ca.jpg,powdery_mildew
2,85f8cb619c66b863.jpg,powdery_mildew


In [32]:
print(device, next(model.parameters()).device)
!nvidia-smi

cuda:0 cuda:0
Wed Oct 15 12:55:10 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   72C    P0             31W /   70W |    4855MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+---------------------------------